In [1]:
%matplotlib inline
import os
import sys

#if you cloned from github, uncomment below!
sys.path.append("../")

from sapsan.lib.backends.fake import FakeExperimentBackend
from sapsan.lib.backends.mlflow import MlFlowExperimentBackend
from sapsan.lib.data.jhtdb_dataset import JHTDB128Dataset
from sapsan.lib.data import Equidistance3dSampling
from sapsan.lib.estimator import Spacial3dEncoderNetworkEstimator, Spacial3dEncoderNetworkEstimatorConfiguration
from sapsan.lib.experiments.evaluation_3d import Evaluation3dExperiment
from sapsan.lib.experiments.training import TrainingExperiment

/home/pkarpov/.conda/envs/torch/lib/python3.7/site-packages/pywt/_utils.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/home/pkarpov/.conda/envs/torch/lib/python3.7/site-packages/catalyst/utils/registry.py:21: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working

/home/pkarpov/.conda/envs/torch/lib/python3.7/site-packages/wandb/wandb_config.py:1: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working

/home/pkarpov/.conda/envs/torch/lib/python3.7/site-packages/watchdog/utils/bricks.py:175: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working



In [2]:
#--- Experiment tracking backend ---

os.environ["AWS_ACCESS_KEY_ID"] = "<AWS_ACCESS_KEY_ID>"
os.environ["AWS_SECRET_ACCESS_KEY"] = "<AWS_SECRET_ACCESS_KEY>"

experiment_name = "CNN experiment"

#Fake (disable backend)
tracking_backend = FakeExperimentBackend(experiment_name)

#MLflow
#launch mlflow with: mlflow ui --port=9000
#uncomment tracking_backend to use mlflow

MLFLOW_BACKEND_HOST = "localhost"
MLFLOW_BACKEND_PORT = 9000

#tracking_backend = MlFlowExperimentBackend(experiment_name, MLFLOW_BACKEND_HOST, MLFLOW_BACKEND_PORT)

In [3]:
#--- Data setup ---
#In the intereset of loading and training multiple timesteps
#one can specify which checkpoints to use and where
#they appear in the path via syntax: {checkpoint:format}
#
#Next, you need to specify which features to load; let's assume 
#        path = "{feature}.h5"
#
# 1) If in different files, then specify features directly
#    Ex: features = ['velocity', 'denisty', 'pressure']
# 2) If in the same, then duplicate the name in features
#    and specify which labels to pull
#    Ex: features = ["data", "data", "data"]
#        feature_labels = ['velocity', 'density', 'pressure']

path = "data/t{checkpoint:1.0f}/{feature}_dim32_fm15.h5"
features = ['u']
target = ['u']

#Dimensionality of your data per axis
CHECKPOINT_DATA_SIZE = 32

#Reduce dimensionality of each axis to
SAMPLE_TO = 16

#Dimensionality of each axis in a batch
GRID_SIZE = 8

#Sampler to use for reduction
sampler = Equidistance3dSampling(CHECKPOINT_DATA_SIZE, SAMPLE_TO)

In [ ]:
#--- Train the model ---
#Machine Learning model to use
estimator = Spacial3dEncoderNetworkEstimator(
    config=Spacial3dEncoderNetworkEstimatorConfiguration(n_epochs=10, grid_dim=GRID_SIZE)
)

#Load the data
x, y = JHTDB128Dataset(path=path,
                       features=features,
                       target=target,
                       checkpoints=[0],
                       grid_size=GRID_SIZE,
                       checkpoint_data_size=CHECKPOINT_DATA_SIZE,
                       sampler=sampler).load()

#Set the experiment
training_experiment = TrainingExperiment(name=experiment_name,
                                         backend=tracking_backend,
                                         model=estimator,
                                         inputs=x, targets=y)
#Train the model
training_experiment.run()

<IPython.core.display.Javascript object>

In [ ]:
#--- Test the model ---
#Load the test data
x, y = JHTDB128Dataset(path=path,
                       features=features,
                       target=target,
                       checkpoints=[0],
                       grid_size=GRID_SIZE,
                       checkpoint_data_size=CHECKPOINT_DATA_SIZE,
                       sampler=sampler).load()

#Set the test experiment
evaluation_experiment = Evaluation3dExperiment(name=experiment_name,
                                               backend=tracking_backend,
                                               model=training_experiment.model,
                                               inputs=x, targets=y,
                                               grid_size=GRID_SIZE,
                                               checkpoint_data_size=SAMPLE_TO)

#Test the model
evaluation_experiment.run()